In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

In [14]:
df_scores = pd.read_csv("similarity_scores.csv")
df_scores.head()

,Unnamed: 0,product_name,product_review,similarity_score
0,1,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,0.035921
1,2,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,0.019702
2,3,Kentucky Brunch Brand Stout,2021 vintage bottle 79\n\nHoly Fucking Shit KB...,0.110674
3,4,Kentucky Brunch Brand Stout,Celebrating my buddy Rug with his 1000th beer ...,0.071157
4,5,Kentucky Brunch Brand Stout,Thick and syrupy pour mocha head Aroma is booz...,0.063246


In [15]:
def windowmaker(attr, sent):
    if attr not in sent:
        return None
    attr_idx = sent.index(attr)
    i, j = attr_idx - 2, attr_idx + 2
    if i < 0:
        i = 0
    if j > len(sent):
        j = len(sent)
    return sent[i:j]

In [16]:
stop = stopwords.words('english')

def remove_stopwords(sent):
    new = []
    for w in sent:
        if w not in stop:
            new.append(w)
    return new

In [17]:
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_score(sentence):
    if not sentence:
        return None
    sentence = ' '.join(sentence)
    score = analyser.polarity_scores(sentence)
    return score['compound']

In [18]:
attrs = ['chocolate', 'dark', 'sweet', 'bourbon', 'coffee']

df_scores_filt = df_scores[df_scores['similarity_score'] != 0].copy()

for attr in attrs:
    col_name = f'{attr}_score'
    df_scores_filt[col_name] = df_scores_filt['product_review'].apply(str.lower)
    df_scores_filt[col_name] = df_scores_filt[col_name].apply(lambda r: re.sub(r'[^A-Za-z0-9 ]+', '', r))
    df_scores_filt[col_name] = df_scores_filt[col_name].apply(str.split)
    df_scores_filt[col_name] = df_scores_filt[col_name].apply(remove_stopwords)
    df_scores_filt[col_name] = df_scores_filt[col_name].apply(lambda r: windowmaker(attr, r))
    df_scores_filt[col_name] = df_scores_filt[col_name].apply(lambda r: sentiment_analyzer_score(r))

In [19]:
df_scores_filt.head()

,Unnamed: 0,product_name,product_review,similarity_score,chocolate_score,dark_score,sweet_score,bourbon_score,coffee_score
0,1,Kentucky Brunch Brand Stout,I didnt think i was going to give it a perfect...,0.035921,NaN,NaN,NaN,NaN,0.0000
1,2,Kentucky Brunch Brand Stout,So I just read a review that called the legend...,0.019702,NaN,0.6222,NaN,NaN,0.1779
2,3,Kentucky Brunch Brand Stout,2021 vintage bottle 79\n\nHoly Fucking Shit KB...,0.110674,0.0,NaN,NaN,0.0,0.0000
3,4,Kentucky Brunch Brand Stout,Celebrating my buddy Rug with his 1000th beer ...,0.071157,0.0,NaN,NaN,NaN,NaN
4,5,Kentucky Brunch Brand Stout,Thick and syrupy pour mocha head Aroma is booz...,0.063246,NaN,NaN,NaN,NaN,NaN


In [20]:
df_scores_filt.describe()

,Unnamed: 0,similarity_score,chocolate_score,dark_score,sweet_score,bourbon_score,coffee_score
count,989.000000,989.000000,496.000000,471.000000,468.000000,368.000000,347.000000
mean,1294.995956,0.114104,0.094939,0.077446,0.473938,0.112040,0.080115
std,762.544032,0.079937,0.215531,0.195770,0.210252,0.221719,0.209191
min,1.000000,0.007223,-0.557400,-0.493900,-0.471700,-0.476700,-0.526700
25%,592.000000,0.052154,0.000000,0.000000,0.458800,0.000000,0.000000
50%,1359.000000,0.093026,0.000000,0.000000,0.458800,0.000000,0.000000
75%,1982.000000,0.156293,0.000000,0.000000,0.509500,0.177900,0.000000
max,2504.000000,0.454077,0.807400,0.807400,0.859100,0.875000,0.784500


In [23]:
df_brandwise = df_scores_filt[df_scores_filt['similarity_score'] != 0].groupby(by='product_name').mean()

In [31]:
def avg(row):
    scores = [row['chocolate_score'], row['dark_score'], row['sweet_score'], row['bourbon_score'], row['coffee_score']]
    n = 1 # war crime
    avg = 0
    for s in scores:
        if np.isnan(s) or s==0:
            continue
        n += 1
        avg += s
    return avg / n

df_brandwise['avg_score'] = df_brandwise.apply(lambda b: avg(b), axis=1)

In [39]:
df_brandwise.head()

,Unnamed: 0,similarity_score,chocolate_score,dark_score,sweet_score,bourbon_score,coffee_score,avg_score
product_name,,,,,,,,
A Deal With The Devil - Double Oak-Aged,197.75,0.095450,NaN,0.000000,0.48415,0.075433,0.0000,0.186528
A Deal With The Devil - Triple Oak-Aged,177.50,0.138721,NaN,0.000000,0.45880,0.000000,NaN,0.229400
Abner,392.00,0.023313,NaN,NaN,0.45880,NaN,NaN,0.229400
Abrasive Ale,1741.00,0.015972,NaN,NaN,0.45880,NaN,NaN,0.229400
Abraxas,2393.70,0.095875,0.079629,0.069675,0.53690,0.273200,0.0602,0.169934
